In [1]:
!pip install python-dotenv

In [30]:
import requests
import json
import os
from pprint import pprint as pp
from dotenv import load_dotenv

# load .env
load_dotenv()
IG_ACCESS_TOKEN=os.environ.get("IG_ACCESS_TOKEN")
FB_APP_SECRET_CODE=os.environ.get('FB_APP_SECRET_CODE')
IG_BUSINESS_ID=os.environ.get('IG_BUSINESS_ID')

# FB_APP_ID=os.environ.get('FB_APP_ID')
# FB_PAGE_ID=os.environ.get('FB_PAGE_ID')
# IG_USER_ID=os.environ.get("IG_USER_ID")
# FB_USER_ID=os.environ.get('FB_USER_ID')

def getCreds():
    """
    Get creds required for use in the applications

    Returns:
        dictionary: credentials needed globally
    """

    creds = dict() 
    creds['access_token'] = IG_ACCESS_TOKEN   # access token for use with all api calls
    creds['client_secret'] = FB_APP_SECRET_CODE  # client secret from facebook app
    creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
    creds['graph_version'] = 'v20.0' # version of api we are hitting
    creds['instagram_account_id'] = IG_BUSINESS_ID # user business account id

    # with domain and version
    creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint

    # creds['client_id'] = FB_APP_ID  # client id from facebook app IG graph API Test
    # creds['ig_username'] = 'analyst_hyuk' # ig usernaem
    # creds['page_id'] = FB_PAGE_ID # users page id
    # creds['debug'] = 'no' # debug mode for api call

    return creds
    

In [31]:
def makeApiCall(url, endpointParams, type):
    """ Request data from endpoint with params

    Args:
        url: string of the url endpoint to make request from
        endpointParams: dictionary keyed by the names of the url parameters

    Return:
        object: data from the endpoint
    """

    if type == 'POST': # post request
        data = requests.post(url, endpointParams)
    else: # get request
        data = requests.get(url, endpointParams)
    
    data = requests.get(url, endpointParams) # make get request

    response = dict()
    response['url'] = url
    response['endpoint_params'] = endpointParams
    response['endpoint_params_pretty'] = json.dumps(endpointParams, indent=4) # pretty point for cli
    response['json_data'] = json.loads(data.content) # response data from the api
    response['json_data_pretty'] = json.dumps(response['json_data'], indent=4) # pretty point for cli

    return response

In [14]:
# def getInstagramAccount( params ) :
# 	endpointParams = dict() # parameter to send to the endpoint
# 	endpointParams['access_token'] = params['access_token'] # tell facebook we want to exchange token
# 	endpointParams['fields'] = 'instagram_business_account' # access token
# 	url = params['endpoint_base'] + params['page_id'] # endpoint url
# 	return makeApiCall( url, endpointParams, 'GET') # make the api call

# params = getCreds() # get creds
# params['debug'] = 'yes' # set debug
# response = getInstagramAccount( params ) # get debug info

In [32]:
def createMediaObject(params):
    """Create media object
    
    Args:
        params: dictionary of params

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?video_url={video-url}&caption={caption}&access_token={access_token}

    Returns:
        object: data from the endpoint
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'

    endpointParams = dict()
    endpointParams['caption'] = params['caption']
    endpointParams['access_token'] = params['access_token']

    if params['media_type'] == 'image':
        endpointParams['image_url'] = params['media_url']
    else:
        endpointParams['video_url'] = params['media_url']
        endpointParams['media_type'] = params['media_type']
    
    return makeApiCall(url, endpointParams, 'POST')

In [18]:

# def getMediaObjectStatus(mediaObjectId, params):
#     """ Check the status of a media object

#     Args:
#         mediaObjectId: id of the media obejct
#         params: dictionary of params

#     API Endpoint:
#         https://graph.facebook.com/v20.0/{ig-container-id}?fileds=status_code
    
#     Returns:
#         object: data from the endpoint
#     """

#     url = params['endpoint_base'] + '/' + mediaObjectId

#     endpointParams = dict()
#     endpointParams['fields'] = 'status_code'
#     endpointParams['access_token'] = params['access_token']

#     return makeApiCall(url, endpointParams, 'GET')

def publishMedia(mediaObjectId, params):
    """ Publish content

    Args:
        mediaObjectId: id of the media obejct
        params: dictionary of params

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media_publish?creation_id={creation-id}&access_token={access_token}

    Returns:
        object: data from the endpoint
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media_publish'

    endpointParams = dict()
    endpointParams['creation_id'] = mediaObjectId
    endpointParams['access_token'] = params['access_token']

    return makeApiCall(url, endpointParams, 'POST')

In [20]:
# main.py

params = getCreds()
params['media_type'] = 'IMAGE'
params['media_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCK8YiyxtZvfmGrgrxdwbLG1z7o9VxdI5Nkw&s'
params['caption'] = 'This image was posted through the Instagram Graph API with a python script'

In [22]:
imageMediaObjectResponse = createMediaObject(params)

In [39]:
imageMediaObjectResponse['json_data']['data'][0]['id']

'18050730832696979'

In [37]:
publishMedia(imageMediaObjectId, params)

{'url': 'https://graph.facebook.com/v20.0/17841467510518283/media_publish',
 'endpoint_params': {'creation_id': '18159784981312615',
  'access_token': 'EAAMZCN23AZACwBOzBDKl6e5jNLbebNYC8w9hJfDDobcZAhEmVjD9ZBEIEpf3WHH21mn2HzY3BC2vlRedBrEZA3CbZC1qZBFpZAoEbwjhmgJS14woDN1U6C3qXPbi2ZAsqIpZCtrqOc8v99ZBZBOSNsB1UUY3mmZB1UWbrY7yOWVIFkyAXijnlLCb38pciNzZCm'},
 'endpoint_params_pretty': '{\n    "creation_id": "18159784981312615",\n    "access_token": "EAAMZCN23AZACwBOzBDKl6e5jNLbebNYC8w9hJfDDobcZAhEmVjD9ZBEIEpf3WHH21mn2HzY3BC2vlRedBrEZA3CbZC1qZBFpZAoEbwjhmgJS14woDN1U6C3qXPbi2ZAsqIpZCtrqOc8v99ZBZBOSNsB1UUY3mmZB1UWbrY7yOWVIFkyAXijnlLCb38pciNzZCm"\n}',
 'json_data': {'error': {'message': '(#100) Tried accessing nonexisting field (media_publish) on node type (IGUser)',
   'type': 'OAuthException',
   'code': 100,
   'fbtrace_id': 'ApoVnbUd2kIe3_keCMT-auW'}},
 'json_data_pretty': '{\n    "error": {\n        "message": "(#100) Tried accessing nonexisting field (media_publish) on node type (IGUser)",\n  

In [116]:
media_data = {'image_url':params['media_url'], 'caption':params['caption'], 'access_token':params['access_token']}

response = requests.post(params['endpoint_base']+params['instagram_account_id']+'/media', media_data)
# print(params['endpoint_base']+params['instagram_account_id']+'/media')
pp(response.json())

{'id': '18071294863546010'}


In [28]:
imageMediaObjectId = imageMediaObjectResponse['json_data']['data'][1]['id']
imageMediaObjectStatusCode = 'IN_PROGRESS'

print("\n---- IMAGE MEDIA OBJECT ---- \n") # title
print("\tID:") # label
print("\t" + imageMediaObjectId) # id of the object

publishImageResponse = publishMedia(imageMediaObjectId, params)

print("\n---- PUBLISHED IMAGE RESPONSE ---- \n")
print("\tResponse:")
print(publishImageResponse['json_data_pretty'])


---- IMAGE MEDIA OBJECT ---- 

	ID:
	18159784981312615

---- PUBLISHED IMAGE RESPONSE ---- 

	Response:
{
    "error": {
        "message": "(#100) Tried accessing nonexisting field (media_publish) on node type (IGUser)",
        "type": "OAuthException",
        "code": 100,
        "fbtrace_id": "AACBVajZFgLb0jpJ-xnLDlK"
    }
}


In [ ]:
import requests

access_token = IG_ACCESS_TOKEN
media_id = '465050113145549'

# 게시물 상태 확인
status_url = f'https://graph.facebook.com/v20.0/{media_id}'
params = {
    'fields': 'status_code',
    'access_token': access_token
}
response = requests.get(status_url, params=params)
status_response = response.json()

pp(status_response)

{'error': {'code': 100,
           'fbtrace_id': 'AwYkJ1JvSOY_LqvlIUdUhW5',
           'message': '(#100) Tried accessing nonexisting field (status_code) '
                      'on node type (Business)',
           'type': 'OAuthException'}}


```python
POST https://graph.facebook.com/{api-version}/{ig-user-id}/media
    ?image_url={image-url}
    &is_carousel_item={is-carousel-item}
    &caption={caption}
    &location_id={location-id}
    &user_tags={user-tags}
    &product_tags={product-tags}
    &access_token={access-token}
```

In [5]:
import requests
import pprint
version = 'v20.0'
base_url = 'https://graph.facebook.com/' + version
fields = 'id, name'
media_url = f'{base_url}/{ig_business_id}?fields={fields}&access_token={IG_ACCESS_TOKEN}'

response = requests.get(media_url)
media_data = response.json()
pp(media_data)


NameError: name 'ig_business_id' is not defined

In [6]:
ig_business_id = "17841467510518283"
# fb_id = "352736951262950"

create_media_url = f'{base_url}/{ig_business_id}/media'
media_data = {
    
    "image_url" : 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCK8YiyxtZvfmGrgrxdwbLG1z7o9VxdI5Nkw&s',
    "caption" : "good_test",
    "access_token": IG_ACCESS_TOKEN
}

response = requests.post(create_media_url, data=media_data)
media_response= response.json()

print(media_response)

{'id': '18030268691473731'}


In [7]:
# 미디어 객체 게시
if 'id' in media_response:
    media_id = media_response['id']

    publish_media_url = f'https://graph.facebook.com/v20.0/{ig_business_id}/media_publish'
    publish_data = {
        'creation_id': media_id,
        'access_token': IG_ACCESS_TOKEN
    }

    response = requests.post(publish_media_url, data=publish_data)
    publish_response = response.json()
    print(publish_response)
else:
    print("Failed to create media object:", media_response)

{'id': '18050730832696979'}


In [40]:
18159784981312615


# 미디어 객체 게시

media_id = '18050730832696979'

publish_media_url = f'https://graph.facebook.com/v20.0/{ig_business_id}/media_publish'
publish_data = {
    'creation_id': media_id,
    'access_token': IG_ACCESS_TOKEN
}

response = requests.post(publish_media_url, data=publish_data)
publish_response = response.json()
print(publish_response)


{'error': {'message': 'The requested resource does not exist', 'type': 'OAuthException', 'code': 24, 'error_subcode': 2207008, 'is_transient': False, 'error_user_title': '미디어 빌더를 찾을 수 없습니다', 'error_user_msg': 'creation id = 18050730832696979인 미디어 빌더가 없거나 만료되었습니다.', 'fbtrace_id': 'ACzna0OGEXmgY_ddINaRVlS'}}


In [10]:
from pprint import pprint as pp

response = requests.get(f'{base_url}/{ig_business_id}/content_publishing_limit')
media_data = response.json()
pp(media_data)

{'error': {'code': 200,
           'fbtrace_id': 'A8WTxsA1bjfqlBSxVfOvJUy',
           'message': '(#200) Provide valid app ID',
           'type': 'OAuthException'}}


In [41]:
import requests

base_url = 'https://graph.facebook.com/v20.0'
media_url = f'{base_url}/465050113145549?fields=id,name&access_token={IG_ACCESS_TOKEN}'

response = requests.get(media_url)
media_data = response.json()

print(media_data)

{'id': '465050113145549', 'name': 'analyst_hyuk'}
